In [13]:
import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt

from tqdm import tqdm

In [163]:
D_MAX = 10
VARIABLES = [f'x{i}' for i in range(D_MAX)]
SYMBOLS = list(sp.symbols(' '.join(VARIABLES)))

class Sqr(sp.Function):
    @classmethod
    def eval(cls, x):
        return x ** 2
    
class Inv(sp.Function):
    @classmethod
    def eval(cls, x):
        return 1 / x
    
class Abs(sp.Function):
    @classmethod
    def eval(cls, x):
        return sp.Abs(x)

class Atan(sp.Function):
    @classmethod
    def eval(cls, x):
        return sp.atan(x)

class Asin(sp.Function):
    @classmethod
    def eval(cls, x):
        return sp.asin(x)

class Acos(sp.Function):
    @classmethod
    def eval(cls, x):
        return sp.acos(x)

class Tanh (sp.Function):
    @classmethod
    def eval(cls, x):
        return sp.tanh(x)

symbol_dict = {str(var): var for var in SYMBOLS}
symbol_dict['inv'] = Inv
#symbol_dict['sqr'] = Sqr
symbol_dict['abs'] = Abs
symbol_dict['arctan'] = Atan
symbol_dict['arcsin'] = Asin
symbol_dict['arccos'] = Acos
symbol_dict['tanh'] = Tanh

In [166]:
#use healed files attached

df1 = pd.read_csv('FeynmanEquations.csv')
df2 = pd.read_csv('BonusEquations.csv')
df1 = df1[df1['Formula'].notnull()]
df2 = df2[df2['Formula'].notnull()]

In [167]:
feynman_expressions = []

for index, row in tqdm(df1.iterrows()):
    for i in range (1, int(row['# variables']+1)):
        row['Formula'] = row['Formula'].replace(row[f"v{i}_name"], f"x{i}")
    row['Formula'] = row['Formula'].replace("pi", "3.1415")
    feynman_expressions.append((int(row['Number']), int(row['# variables']), row['Formula']))#, row['Formula'].count('(') - row['Formula'].count(')')))

for index, row in tqdm(df2.iterrows()):
    for i in range (1, int(row['# variables']+1)):
        row['Formula'] = row['Formula'].replace(row[f"v{i}_name"], f"x{i}")
    row['Formula'] = row['Formula'].replace("pi", "3.1415")
    feynman_expressions.append((200+int(row['Number']), int(row['# variables']), row['Formula']))#, row['Formula'].count('(') - row['Formula'].count(')')))

100it [00:00, 6401.17it/s]
20it [00:00, ?it/s]


In [168]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def simplify(number, num_var, expr):
    sympy_expr = sp.sympify(expr, locals=symbol_dict)
    return (number, num_var, sp.simplify(sympy_expr))

def parallel_simplification(feynman_expressions, timeout=3):
    results = []
    with ThreadPoolExecutor() as executor:
        # Submit tasks to generate and evaluate random expressions
        futures = {executor.submit(simplify, number, num_var, expr) : (number, num_var, expr) for (number, num_var, expr) in feynman_expressions}
        for future in futures:
            try:
                result = future.result(timeout=timeout)  # Set the timeout for each thread
                results.append(result)
            except TimeoutError:
                number = futures[future][0]
                print(f"Timeout occurred for expression: {number}")
                results.append(None)  # Append None or any placeholder for timed-out tasks
    return results

In [169]:
if __name__ == '__main__':
    results = parallel_simplification(feynman_expressions, timeout=3)

In [181]:
def inf_to_pref(expr):
    precedence = {
        ' + ': 1, ' - ': 1,
        ' * ': 2, ' / ': 2
    }
    stack = []  # Operator stack
    output = []  # Prefix output

    def precedence_of(op):
        return precedence.get(op, 0)
    
    expr = expr[::-1]  # Reverse the expression for right-to-left traversal

    i = 0
    while i < len(expr) :
        token = expr[i]
        if i < len(expr)-3 and (expr[i]+expr[i+1]+expr[i+2]) in precedence:
            while (stack and precedence_of(stack[-1]) > precedence_of(expr[i]+expr[i+1]+expr[i+2])):
                output.append(stack.pop())
            stack.append(expr[i]+expr[i+1]+expr[i+2])
            output.append(' ')
            i += 3
        elif not token in '()' :
            output.append(token) # Add operand to the output
            i+=1
        elif token == ')':  # Closing parenthesis (reversed input)
            stack.append(token)
            i+=1
        elif token == '(':
            while stack and stack[-1] != ')':
                output.append(stack.pop())
            stack.pop()
            i+=1# Pop the closing parenthesis
                
    while stack:
        output.append(stack.pop())

    return ''.join(output[::-1]).split()  # Reverse the output for prefix format

In [197]:
pref_results = []

for (number, num_var, expr) in results :
    expr = str(expr)
    expr = expr.replace('(', '( ')
    expr = expr.replace(')', ' )')
    expr = expr.replace('*', ' * ')
    expr = expr.replace('/', ' / ')
    pref_expr = inf_to_pref(expr)
    pref_results.append((number, num_var, pref_expr))

In [199]:
results[0], pref_results[0]

((1, 1, 0.398948163444861*exp(-0.5*sqr(x1))),
 (1, 1, ['*', '0.398948163444861', 'exp', '*', '-0.5', 'sqr', 'x1']))